In [2]:
# 패키지 불러오기
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils

In [33]:
# 랜덤시드
np.random.seed(5)

#4분의 4박자
N = 4 

# loss 함수 정의
class LossHistory(keras.callbacks.Callback):
    def init(self):
        self.losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

# training data set만들기
# 4개씩 끊어서 이를 numpy를 이용해 배열로 각 저장해 출력
def seq2dataset(seq, window_size):
    dataset = []
    for i in range(len(seq)-window_size):
        subset = seq[i:(i+window_size+1)]
        dataset.append([code2idx[item] for item in subset])
    return np.array(dataset)

In [34]:
# Bar : 마디
#Note : 음표, Rest : 쉼표
#Dur 음표의 길이, Pos : 음표의 높이 (Pos의 3번째 값이 기준으로 0을 지님, Dotted : 해당음표의 0.5)
# Stem : 음표의 가지 (Up : 위, Down : 아래)
#Beam : 잇단음표 (음표의 가지가 하나로 연결되게)
# 훈련할 악보데이터 만들기
notes1 = ['Note' , 'Rest']
notes2 = ['Dur:Whole' , 'Dur:Half' , 'Dur:4th' , 'Dur:8th' , 'Dur:16th' , 'Dur:32nd']
notes3 = ['Pos:' + str(i) for i in range(-12,9)] + ['Pos:b0' , 'Pos:#-3' , 'Pos:#-2' , 'Pos:#-1' , 'Pos:#1' , 'Pos:#2' , 'Pos:#4' , 'Pos:n-1' , 'Pos:n-2']
notes4 = ['Dur:Whole' , 'Dur:Half' , 'Dur:4th' , 'Dur:8th' , 'Dur:16th' , 'Dur:32nd']
notes5 = ['Stem=Up' , 'Stem=Down']
notes6 = ['Beam=First' , 'Beam=End']
notes7 = ['' , ',Dotted']
notes8 = ['' , '^']
notes9 = ['' , 'B']

In [35]:
all_cs = []
#여러 요소들을 조합하여 음표사전 만드는 데에 참조할 단어집합 만들기

#음표에 관한 조합 만들기
for s2 in notes2:
    for s3 in notes7:
        for s4 in notes3:
            for s5 in notes8:
                for s6 in notes9:
                    cs = ('|Note|' + s2 + s3 + '|' + s4 + s5 + '|' + s6)
                    all_cs = all_cs + [cs]
                    #print(cs)

#쉼표에 관한 조합 만들기
for s1 in notes4:
    for s2 in notes7:
        cs = ('|Rest|' + s1 + s2)
        all_cs = all_cs + [cs]
        #print(cs) 

#리스트 끝에 '|'가 존재할 시 제거해주는 작업
all_cs2 = []
for s in all_cs:
    if s[len(s)-1] == '|':
        s = s[:-1]
    all_cs2.append(s)
all_cs = all_cs2
    
#음표사전 만들기 
idx = [idx for idx in range(0,len(all_cs))]
idx2code = dict(zip(idx , all_cs)) # index to code
code2idx = dict(zip(all_cs , idx)) # code to index

In [36]:
# 사전의 길이 확인
len(all_cs) , len(idx2code) , len(code2idx)

(1452, 1452, 1452)

In [37]:
import codecs
from bs4 import BeautifulSoup

#트레이닝할 악보 읽어오기(가요 '목로주점'외 10곡)
fp = codecs.open("NOTES.txt", "r")
lines = fp.readlines()
fp.close()

# 전처리하기
# 이전에 만든 단어 사전의 형태와 일치시켜주기 위한 작업
# 변경 전 : |Note|Dur:16th|Pos:-4|Opts:Stem=Up,Beam=First
# 변경 후 : |Note|Dur:16th|Pos:-4|B
all_lines = [] 

for str1 in lines:
    if not bool(str1.find('Bar')+1) and not bool(str1.find('Chord')+1):
        str1 = str1.replace("\n","")
        str1 = str1.replace("Stem=Up,","")
        str1 = str1.replace("Stem=Down,","")
        str1 = str1.replace("Opts:Beam=First","B")
        str1 = str1.replace("|Opts:Beam=End","")        
        str1 = str1.replace("Opts:Beam","B")    
        str1 = str1.replace(",Slur","")                
        str1 = str1.replace("|Opts:Stem=Up","")  
        str1 = str1.replace("|Opts:Stem=Down","")  
        str1 = str1.replace(",Triplet=First","")     
        str1 = str1.replace(",Triplet=End","")
        str1 = str1.replace(",Triplet","") 
        all_lines.append(str1)
        #print(str1) 

In [38]:
#LSTM에 넣을 재료 만들기
seq = all_lines
dataset = seq2dataset(seq, window_size = 3)
print(dataset.shape)
print(dataset)

(3064, 4)
[[1440  492 1446  732]
 [ 492 1446  732  512]
 [1446  732  512  512]
 ...
 [ 782  781  772  648]
 [ 781  772  648  776]
 [ 772  648  776   52]]


In [48]:
#input / target 설정하기
max_idx_value = len(idx2code)-1
x_train = dataset[:,0:3] # 4개 중 3개 훈련 데이터
y_train = dataset[:,3] # 4개 중 마지막 1개 정답 데이터

#입력값 정규화
x_train = x_train / float(max_idx_value)

#데이터건수*time step*1
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

#one-hot encoding
#One-hot Encoding이란 단 하나의 값만 True이고 나머지는 모두 False인 인코딩
y_train = np_utils.to_categorical(y_train)
one_hot_vec_size = y_train.shape[1]

x_train.shape , y_train.shape , one_hot_vec_size

((3064, 3, 1), (3064, 1451), 1451)

In [ ]:
#모델 설계
model = Sequential()
model.add(LSTM(128, input_shape = (3, 1)))
model.add(Dense(one_hot_vec_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#loss function
history = LossHistory()
history.init()

#모델 학습시작
model.fit(x_train, y_train, epochs=1000, batch_size=14, verbose=2, callbacks=[history])

In [ ]:
# 트레이닝 과정
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(history.losses)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

# 모델평가
scores = model.evaluate(x_train, y_train)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [49]:
#작곡시작
pred_count = 200 #생성할 note(또는 rest)의 수
    
seq_in = ['|Note|Dur:4th|Pos:-4', '|Note|Dur:4th|Pos:-2', '|Note|Dur:Half,Dotted|Pos:-2']
seq_out = seq_in
seq_in = [code2idx[it] / float(max_idx_value) for it in seq_in] # 코드를 인덱스값으로 변환

for i in range(pred_count):
    sample_in = np.array(seq_in)
    sample_in = np.reshape(sample_in, (1, 3, 1))
    pred_out = model.predict(sample_in)
    idx = np.argmax(pred_out)
    seq_out.append(idx2code[idx])
    seq_in.append(idx / float(max_idx_value))
    seq_in.pop(0)

print("full song prediction : ", seq_out)

[0.3528601  0.35837354 0.27567195]


NameError: name 'model' is not defined

In [ ]:
nwc = open('창작곡.txt', 'w')

for item in seq_out:
  nwc.write("%s\n" % item)

nwc.close()

In [ ]:
# NWC format으로 다시 만들기 
import re
fp = codecs.open("창작곡.txt", "r")
lines = fp.readlines()
len_sum = 0 
length = 0 
bar = 0.25 * N   #마디 1개의 길이
update_lines = []
over_length = 0 
cnt = 0 
fp.close()

In [ ]:
for note in lines:
    
    cnt += 1
    need_special = False
    resid_special = False
    note = note.replace('\n' , '')
    note_comp = note.split('|')
    
    #길이추출
    matching = [s for s in note_comp if "Dur" in s][0].split(":")[1]
    
    #마디를 씌우기 위한 누적계산
    if matching.find('Half')+1:
        length = 0.5
    elif matching.find('Whole')+1:
        length = 1
    else:        
        length = 1/int(re.findall('\d+', matching)[0])
        
    if matching.find('Dotted')+1:
        length = length + length/2
        
    len_sum += length

In [ ]:
#1이하인 경우는 정상이므로 추가한다. 
    if len_sum <= 1:
        update_lines.append(note)
        print(note , length , len_sum)
        
        if len_sum == 1:
            len_sum = 0 
            update_lines.append('|Bar')
            print('|Bar')
        
        #마지막 마디에서 완전하게 음표를 채운다. 
        if len(lines) == cnt:
            print('**********마지막 마디입니다.' , note , len_sum)
            need_len = 1 - len_sum

In [ ]:
if need_len > 0:
                
                #마지막 음표를 채운다. 
                if need_len == 0.75:
                    Dur_str = "Half,Dotted"
                elif need_len == 0.5:
                    Dur_str = "Half"
                elif need_len == 0.375:
                    Dur_str = "4th,Dotted"
                elif need_len == 0.25:
                    Dur_str = "4th"
                elif need_len == 0.1875:
                    Dur_str = "8th,Dotted"
                elif need_len == 0.125:
                    Dur_str = "8th"
                elif need_len == 0.09375:
                    Dur_str = "16th,Dotted"
                elif need_len == 0.0625:
                    Dur_str = "16th"
                elif need_len == 0.046875:
                    Dur_str = "32th,Dotted"
                elif need_len == 0.03125:
                    Dur_str = "32th"
                
                note_comp = note.split('|')
                index = [i for i, string in enumerate(note_comp) if "Dur" in string][0]
                Dur_str = "Dur:" + Dur_str
                
                note_comp[index] = Dur_str
                need_str = '|'.join(note_comp)
                
                #마지막 마디의 마지막 음표를 추가한다.(쉼표가 될 수도 있음)
                if Dur_str != "Dur:":
                    update_lines.append(need_str)
                    print("*" , need_str , need_len , 1.0)

In [ ]:
else:
        #길이가 over가 된 경우에는 음표를 2개로 쪼개고 가운데에 bar을 찍는다. 
        #Dur이 속한 위치와 길이를 알아낸다.(Dur를 수정해야 하기 때문에)
        note_comp = note.split('|')
        index = [i for i, string in enumerate(note_comp) if "Dur" in string][0]
        matching = [s for s in note_comp if "Dur" in s][0].split(":")[1]
        
        if "Whole" in matching:
            over_length = 1
        elif "Half" in matching:
            over_length = 0.5
        elif "4th" in matching:
            over_length = 1/4
        elif "8th" in matching:
            over_length = 1/8
        elif "16th" in matching:
            over_length = 1/16
        elif "32nd" in matching:
            over_length = 1/32
        else:
            over_length = 0
            
        if "Dotted" in matching:
            over_length = over_length + over_length/2
        
        #마디의 모자란 길이와 남는 길이를 알아내고 bar(마디)를 찍을 준비를 한다. 
        resid_len = len_sum - 1                 #다음 마디로 넘어갈 길이
        need_len = over_length - resid_len      #현재 마디에서 필요한 길이

In [ ]:
 #마디의 남은 부분 채우기(현재마디의 여분) 
        if need_len == 0.75:
            Dur_str = "Half,Dotted"
        elif need_len == 0.5:
            Dur_str = "Half"
        elif need_len == 0.375:
            Dur_str = "4th,Dotted"
        elif need_len == 0.25:
            Dur_str = "4th"
        elif need_len == 0.1875:
            Dur_str = "8th,Dotted"
        elif need_len == 0.125:
            Dur_str = "8th"
        elif need_len == 0.09375:
            Dur_str = "16th,Dotted"
        elif need_len == 0.0625:
            Dur_str = "16th"
        elif need_len == 0.046875:
            Dur_str = "32th,Dotted"
        elif need_len == 0.03125:
            Dur_str = "32th"
        elif need_len == 0:
            Dur_str = ""
        else:
            #해당사항이 없으면 가장 가까운 음표를 찾아서 넣어준다.(이런 경우는 2개이상의 음표가 필요하다.)
            if need_len == 0.15625 or need_len == 0.625:
                need_special = True

            
        #마디의 over된 부분 채우기(다음 마디로 넘어감)
        if resid_len == 0.75:
            Dur_str2 = "Half,Dotted"
        elif resid_len == 0.5:
            Dur_str2 = "Half"
        elif resid_len == 0.375:
            Dur_str2 = "4th,Dotted"
        elif resid_len == 0.25:
            Dur_str2 = "4th"
        elif resid_len == 0.1875:
            Dur_str2 = "8th,Dotted"
        elif resid_len == 0.125:
            Dur_str2 = "8th"
        elif resid_len == 0.09375:
            Dur_str2 = "16th,Dotted"
        elif resid_len == 0.0625:
            Dur_str2 = "16th"
        elif resid_len == 0.046875:
            Dur_str2 = "32th,Dotted"
        elif resid_len == 0.03125:
            Dur_str2 = "32th"
        elif resid_len == 0:
            Dur_str2 = ""
        else:
            #해당사항이 없으면 가장 가까운 음표를 찾아서 넣어준다.
            resid_special = True
        
        #마디의 여분을 채우는 작업 ########################################1
        len_sum = len_sum - resid_len
        
        Dur_str = "Dur:" + Dur_str
        Dur_str2 = "Dur:" + Dur_str2 
        
        note_comp[index] = Dur_str
        need_str = '|'.join(note_comp)

        note_comp[index] = Dur_str2
        resid_str = '|'.join(note_comp)
        
        if Dur_str != "Dur:":
            update_lines.append(need_str)
            print("*" , need_str , need_len , len_sum)
            
        #마디가 맞지 않는 특별한 상황    
        if need_special == True:
            if need_len == 0.15625:    #8분음표와 32분음표를 추가 
                Dur_str = "Dur:8th"
                note_comp[index] = Dur_str
                need_str = '|'.join(note_comp)
                update_lines.append(need_str)
                print("**" , need_str , 1/8 , len_sum)
                
                Dur_str = "Dur:32nd"
                note_comp[index] = Dur_str
                need_str = '|'.join(note_comp)
                update_lines.append(need_str)
                print("**" , need_str , 1/32 , len_sum)
                
            if need_len == 0.625:      #이분음표와 8분음표를 추가
                Dur_str = "Dur:Half"
                note_comp[index] = Dur_str
                need_str = '|'.join(note_comp)
                update_lines.append(need_str)
                print("**" , need_str , 1/2 , len_sum)
                
                Dur_str = "Dur:8th"
                note_comp[index] = Dur_str
                need_str = '|'.join(note_comp)
                update_lines.append(need_str)
                print("**" , need_str , 1/8 , len_sum)
                
            # ....계속 추가하여 완벽하게 만든다. 

In [ ]:
#마디의 경계를 그려준다.(칸막이) ###############################################2
        update_lines.append('|Bar')
        print('|Bar') 

        #다음 마디로 넘어가는 부분을 그려준다. ##########################################3 
        len_sum = resid_len
        
        if Dur_str2 != "Dur:":
            update_lines.append(resid_str)
            print("*" , resid_str , resid_len , len_sum)

        #음표길이 문자열을 초기화한다. ####################################
        Dur_str = ""
        Dur_str2 = ""

In [ ]:
#마디의 경계를 그려준다.(칸막이) ###############################################2
        update_lines.append('|Bar')
        print('|Bar') 

        #다음 마디로 넘어가는 부분을 그려준다. ##########################################3 
        len_sum = resid_len
        
        if Dur_str2 != "Dur:":
            update_lines.append(resid_str)
            print("*" , resid_str , resid_len , len_sum)

        #음표길이 문자열을 초기화한다. ####################################
        Dur_str = ""
        Dur_str2 = ""

In [ ]:
#마지막 라인도 같은 방법으로 수정함.
last_line = update_lines[len(update_lines)-1]
last_line

if 'BF' in last_line:
    last_line = last_line.replace('|BF' , '')
    update_lines[len(update_lines)-1] = last_line
elif 'BM' in last_line:
    last_line = last_line.replace('BM' , 'BE')
    update_lines[len(update_lines)-1] = last_line
    
#print(update_lines)


In [ ]:
cnt = 0 

#특정 작곡프로그램의 format에 맞게 수정함. 
for uline in update_lines:
    cnt += 1
    
    if 'Pos' in uline:
        #음의 높이를 구하기(음의 높이에 따라 stem의 방향이 바뀐다.)
        uline_comp = uline.split('|')
        index = [i for i, string in enumerate(uline_comp) if "Pos" in string][0]
        matching = [s for s in uline_comp if "Pos" in s][0].split(":")[1]
        pos = int(re.findall('[+-]*\d+', matching)[0])
        
        if pos < 0:
            direct = "Opts:Stem=Up,"
        else:
            direct = "Opts:Stem=Down,"
            
    else:
        direct = ""

    uline = uline.replace('BE',direct + 'Beam=End')
    uline = uline.replace('BF',direct + 'Beam=First')
    uline = uline.replace('BM','Beam')

    update_lines[cnt-1] = uline 
    
    print(uline , pos) 

In [ ]:
#최종 악보만들기 

preface = ['!NoteWorthyComposer(2.75)' , '|Clef|Type:Treble' , '|TimeSig|Signature:4/4' , '|Tempo|Tempo:128|Pos:6','|Bar']
postface = ['!NoteWorthyComposer-End']        
update_lines = preface + update_lines + postface

nwc = open('창작곡.nwctxt', 'w')

for uline in update_lines:
  nwc.write("%s\n" % uline)

nwc.close()
